# Загрузка модулей

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import sys
sys.path.append('./src')

In [ ]:
from ipy_utils import (IPYInteractiveSegmentation, show_video,
                       IPYRadioButtons, IPYButton)
from sam2al import SAM2
from cvat import CVATLabeles
from utils import TimeIt

# Открытие видеофайла и списка классов

In [ ]:
video_file = './project/videos/14_fragment_6.avi'

with TimeIt('загрузку модели') as t:
    sam2 = SAM2('./models/sam2.1_hiera_large.pt')

with TimeIt('загрузку видео') as t:
    sam2.load_video(video_file,
                    prompts_file='./project/prompts',
                    offload_video_to_cpu=True,
                    offload_state_to_cpu=False,
                    async_loading_frames=False)

In [ ]:
labeles = CVATLabeles('./project/labeles.txt')
IPYRadioButtons(**labeles.get_ipy_radio_buttons_kwargs('Класс объекта:'),
                setter=sam2.set_label).show();

# Интерактивная сегментация

In [ ]:
sam2.go_to(frame=(5*60+20)*30 * 0, obj_id='new')
ipis = IPYInteractiveSegmentation(**sam2.init_ipis_kwargs())

# Таблица подсказок

In [ ]:
sam2.prompts.df

# Группировка определённых классов:

In [ ]:
for obj_id in sam2.prompts.get_all_obj_ids():
    if sam2.prompts.get_label(obj_id) not in ['People', 'transport']:
        sam2.prompts.set_group(obj_id, 1)
sam2.prompts.make_auto_save()

# Сборка превью

In [ ]:
with TimeIt('формирование простого превью') as t:
    video = show_video(sam2.RenderPreview(), size=480)
video

In [ ]:
with TimeIt('формирование превью с подгонкой контуров') as t:
    video = show_video(sam2.RenderPreview(fit_segments=True), size=480)
video

# Создание файла CVAT-разметки

In [ ]:
with TimeIt('формирование файла-разметки') as t:
    sam2.save2cvat_xml();